In [30]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, VotingRegressor, BaggingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [31]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('sample_submission.csv')
bookings_data = pd.read_csv('bookings_data.csv')
bookings = pd.read_csv('bookings.csv')
hotel_data = pd.read_csv('hotels_data.csv')
customer_data = pd.read_csv('customer_data.csv')
payments_data = pd.read_csv('payments_data.csv')

In [32]:
# keep only entries with payment_sequential as 1
payments_data_unique = payments_data[payments_data['payment_sequential'] == 1]
payments_data_repeat = payments_data[payments_data['payment_sequential'] != 1]
payments_data_unique.describe(include='all')

,booking_id,payment_sequential,payment_type,payment_installments,payment_value
count,99360,99360.0,99360,99360.000000,99360.000000
unique,99360,NaN,5,NaN,NaN
top,6f3fe1789b1e8b2acac839d17b81ef22,NaN,credit_card,NaN,NaN
freq,1,NaN,76476,NaN,NaN
mean,NaN,1.0,NaN,2.929640,158.336774
std,NaN,0.0,NaN,2.714947,220.511857
min,NaN,1.0,NaN,1.000000,0.000000
25%,NaN,1.0,NaN,1.000000,59.947500
50%,NaN,1.0,NaN,2.000000,103.330000
75%,NaN,1.0,NaN,4.000000,175.110000


In [33]:
# sort payments_data_repeat by payment_sequential
payments_data_repeat = payments_data_repeat.sort_values(by=['payment_sequential'], ascending=True)
payments_data_repeat.describe(include='all')

,booking_id,payment_sequential,payment_type,payment_installments,payment_value
count,4526,4526.000000,4526,4526.000000,4526.000000
unique,3039,NaN,4,NaN,NaN
top,d1b0e818e3ccc5cb0e39231352fa65da,NaN,voucher,NaN,NaN
freq,28,NaN,4154,NaN,NaN
mean,NaN,3.127265,NaN,1.178524,61.098164
std,NaN,2.670762,NaN,0.989643,96.149183
min,NaN,2.000000,NaN,0.000000,0.000000
25%,NaN,2.000000,NaN,1.000000,14.905000
50%,NaN,2.000000,NaN,1.000000,31.920000
75%,NaN,3.000000,NaN,1.000000,71.230000


In [34]:
# making payment data unique for each booking_id by adding the payments made by other methods to primary payment method
for payment_data_repeat in payments_data_repeat.itertuples():
    booking_id = payment_data_repeat.booking_id
    payment_value_new = payments_data_unique[payments_data_unique['booking_id'] == booking_id]['payment_value'] + payment_data_repeat.payment_value
    payment_installments_new = payments_data_unique[payments_data_unique['booking_id'] == booking_id]['payment_installments'] + payment_data_repeat.payment_installments
    payments_data_unique.loc[payments_data_unique['booking_id'] == booking_id, 'payment_value'] = payment_value_new
    payments_data_unique.loc[payments_data_unique['booking_id'] == booking_id, 'payment_installments'] = payment_installments_new
    payments_data_unique.loc[payments_data_unique['booking_id'] == booking_id, 'payment_sequential'] = payment_data_repeat.payment_sequential

payments_data_unique.describe(include='all')

,booking_id,payment_sequential,payment_type,payment_installments,payment_value
count,99360,99360.000000,99360,99360.000000,99360.000000
unique,99360,NaN,5,NaN,NaN
top,6f3fe1789b1e8b2acac839d17b81ef22,NaN,credit_card,NaN,NaN
freq,1,NaN,76476,NaN,NaN
mean,NaN,1.044726,NaN,2.981310,160.984584
std,NaN,0.381293,NaN,2.741804,221.998050
min,NaN,1.000000,NaN,1.000000,0.000000
25%,NaN,1.000000,NaN,1.000000,62.010000
50%,NaN,1.000000,NaN,2.000000,105.290000
75%,NaN,1.000000,NaN,4.000000,176.932500


In [35]:
payments_data_unique.drop(['payment_type'], axis=1, inplace=True)
table_1 = pd.merge(bookings_data,hotel_data, on='hotel_id', how='left')
table_1.drop(['hotel_id','seller_agent_id','hotel_category'], axis=1, inplace=True)

In [36]:
date_columns = ['booking_expiry_date']

for date_column in date_columns:
    # convert date to datetime
    table_1[date_column] = pd.to_datetime(table_1[date_column])
    # change the date time entries of column to number of seconds from base date
    table_1[date_column] = table_1[date_column].astype(np.int64) // 10**9  
    # take the average date as the start date
    start_date_seconds =table_1[date_column].mean()
    # subtract the start date from the date column
    table_1[date_column] = table_1[date_column] - start_date_seconds

print(table_1.dtypes)
table_1.describe(include='all')

booking_id                   object
booking_sequence_id           int64
booking_expiry_date         float64
price                       float64
agent_fees                  float64
hotel_name_length           float64
hotel_description_length    float64
hotel_photos_qty            float64
dtype: object


,booking_id,booking_sequence_id,booking_expiry_date,price,agent_fees,hotel_name_length,hotel_description_length,hotel_photos_qty
count,112650,112650.000000,1.126500e+05,112650.000000,112650.000000,111047.000000,111047.000000,111047.000000
unique,98666,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,f5f79c56e9e4120aec44ef8272b63d03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.197834,-5.719929e-08,120.653739,19.990320,48.775978,787.867029,2.209713
std,NaN,0.705124,1.320182e+07,183.633928,15.806405,10.025581,652.135608,1.721438
min,NaN,1.000000,-4.109528e+07,0.850000,0.000000,5.000000,4.000000,1.000000
25%,NaN,1.000000,-9.398365e+06,39.900000,13.080000,42.000000,348.000000,1.000000
50%,NaN,1.000000,1.635763e+06,74.990000,16.260000,52.000000,603.000000,1.000000
75%,NaN,1.000000,1.062343e+07,134.900000,21.150000,57.000000,987.000000,3.000000


In [37]:
# split bookings_data into unique and repeat bookings
bookings_data_unique =table_1[table_1['booking_sequence_id'] == 1]
bookings_data_repeat =table_1[table_1['booking_sequence_id'] > 1]

bookings_data_unique.describe(include='all')

,booking_id,booking_sequence_id,booking_expiry_date,price,agent_fees,hotel_name_length,hotel_description_length,hotel_photos_qty
count,98666,98666.0,9.866600e+04,98666.000000,98666.000000,97250.000000,97250.000000,97250.000000
unique,98666,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,242fe8c5a6d1ba2dd792cb1621400010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.0,-1.047885e+04,125.964327,20.201927,48.846386,794.161398,2.250591
std,NaN,0.0,1.322097e+07,191.375106,15.909873,9.999239,654.751953,1.747095
min,NaN,1.0,-4.109528e+07,0.850000,0.000000,5.000000,4.000000,1.000000
25%,NaN,1.0,-9.463282e+06,41.505000,13.310000,42.000000,349.000000,1.000000
50%,NaN,1.0,1.635331e+06,79.000000,16.360000,52.000000,607.000000,2.000000
75%,NaN,1.0,1.062561e+07,139.900000,21.230000,57.000000,996.000000,3.000000


In [38]:
# sort bookings_data_repeat by booking_sequence_id
bookings_data_repeat = bookings_data_repeat.sort_values(by=['booking_sequence_id'], ascending=True)
bookings_data_repeat.describe(include='all')

,booking_id,booking_sequence_id,booking_expiry_date,price,agent_fees,hotel_name_length,hotel_description_length,hotel_photos_qty
count,13984,13984.000000,1.398400e+04,13984.000000,13984.000000,13797.000000,13797.000000,13797.000000
unique,9803,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,f5f79c56e9e4120aec44ef8272b63d03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.593678,7.393495e+04,83.184167,18.497299,48.279698,743.500326,1.921577
std,NaN,1.334476,1.306614e+07,107.874536,14.971970,10.195912,631.633730,1.497150
min,NaN,2.000000,-4.101272e+07,0.850000,0.000000,6.000000,8.000000,1.000000
25%,NaN,2.000000,-8.767644e+06,30.000000,11.850000,42.000000,338.000000,1.000000
50%,NaN,2.000000,1.901641e+06,56.000000,15.560000,51.000000,557.000000,1.000000
75%,NaN,3.000000,1.060551e+07,99.900000,20.160000,57.000000,919.000000,2.000000


In [ ]:
# merging bookings_data for each booking_id
for booking_data_repeat in bookings_data_repeat.itertuples():
    bookings_id = booking_data_repeat.booking_id
    booking_data_unique = bookings_data_unique[bookings_data_unique['booking_id'] == bookings_id]
    new_price = booking_data_repeat.price + booking_data_unique.price
    new_hlength=booking_data_repeat.hotel_name_length+booking_data_unique.hotel_name_length
    new_hdesc=booking_data_repeat.hotel_description_length+booking_data_unique.hotel_description_length
    new_hqty=booking_data_repeat.hotel_photos_qty+booking_data_unique.hotel_photos_qty 
    new_agent_fees = booking_data_repeat.agent_fees + booking_data_unique.agent_fees
    new_exp=booking_data_repeat.booking_expiry_date + booking_data_unique.booking_expiry_date
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'booking_expiry_date'] = new_exp
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'hotel_name_length'] = new_hlength
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'hotel_description_length'] = new_hdesc
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'hotel_photos_qty'] = new_hqty
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'price'] = new_price
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'agent_fees'] = new_agent_fees
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'booking_sequence_id'] = booking_data_repeat.booking_sequence_id


In [ ]:
for booking_data_unique in bookings_data_unique.itertuples():
    bookings_id = booking_data_unique.booking_id
    booking_data_unique = bookings_data_unique[bookings_data_unique['booking_id'] == bookings_id]
    if bookings_id in bookings_data_repeat['booking_id'].values:
      cnt=1+(bookings_data_repeat['booking_id'].value_counts()[bookings_id])
    else:
      cnt=1
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'hotel_name_length'] = (booking_data_unique.hotel_name_length)/cnt
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'hotel_description_length'] = (booking_data_unique.hotel_description_length)/cnt
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'hotel_photos_qty'] = (booking_data_unique.hotel_photos_qty)/cnt
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'booking_expiry_date'] = (booking_data_unique.booking_expiry_date)/cnt

In [ ]:
bookings_data_unique.describe(include='all')

In [ ]:
# merge bookings and bookings_data as bookings_df
bookings_df = pd.merge(bookings, bookings_data_unique, on='booking_id', how='left')

bookings_df.describe(include='all')

In [ ]:
# merge bookings_df and customer_data as bookings_customer_df
table_2= pd.merge(bookings_df, customer_data, on='customer_id', how='left')

# merge bookings_hotel_df and payments_data as bookings_payment_df
bookings_payment_df = pd.merge(table_2, payments_data_unique, on='booking_id', how='left')

bookings_payment_df.drop(['customer_id'], axis=1, inplace=True)

bookings_payment_df.describe(include='all')

In [ ]:
date_columns = ['booking_create_timestamp','booking_approved_at','booking_checkin_customer_date' ]

for date_column in date_columns:
    # convert date to datetime
    bookings_payment_df[date_column] = pd.to_datetime(bookings_payment_df[date_column])
    # change the date time entries of column to number of seconds from base date
    bookings_payment_df[date_column] = bookings_payment_df[date_column].astype(np.int64) // 10**9  
    # take the average date as the start date
    start_date_seconds =bookings_payment_df[date_column].mean()
    # subtract the start date from the date column
    bookings_payment_df[date_column] = bookings_payment_df[date_column] - start_date_seconds

print(bookings_payment_df.dtypes)
bookings_payment_df.describe(include='all')

In [ ]:
for booking_data_repeat in bookings_payment_df.itertuples():
    bookings_id = booking_data_repeat.booking_id
    booking_data_unique = bookings_payment_df[bookings_payment_df['booking_id'] == bookings_id]
    bookings_payment_df.loc[bookings_payment_df['booking_id'] == bookings_id, 'booking_create_timestamp'] = (booking_data_unique.booking_approved_at)-(booking_data_unique.booking_create_timestamp)
    bookings_payment_df.loc[bookings_payment_df['booking_id'] == bookings_id, 'booking_checkin_customer_date'] = (booking_data_unique.booking_expiry_date)-(booking_data_unique.booking_checkin_customer_date) 
bookings_payment_df.drop(['booking_approved_at','booking_expiry_date'], axis=1, inplace=True)
bookings_payment_df.describe(include='all')

In [ ]:
cat_columns = ['booking_status', 'country', 'customer_unique_id']

for column in cat_columns:
    le = LabelEncoder()
    le.fit(bookings_payment_df[column])
    bookings_payment_df[column] = le.transform(bookings_payment_df[column])
    if column == 'booking_status' or column == 'country':
        bookings_payment_df[column] = bookings_payment_df[column] + 1

print(bookings_payment_df.dtypes)

In [ ]:
# take all columns
columns = bookings_payment_df.columns

# remove booking_id
columns = columns.drop(['booking_id'])

# change all null or nan values to mean of respective columns
for column in columns:
    mean = bookings_payment_df[column].mean()
    bookings_payment_df[column].fillna(mean, inplace=True)

In [ ]:
bookings_payment_df.describe(include='all')

In [ ]:
# assert no null values
assert bookings_payment_df.isnull().sum().sum() == 0

In [ ]:
# train_booking_df contains bookings_df with booking_id in train_data
train_booking_df = bookings_payment_df[bookings_payment_df['booking_id'].isin(train_data['booking_id'])]

# create X_train and Y_train
train_booking_df = train_booking_df.sort_values(by=['booking_id'])
X_train = train_booking_df.drop(['booking_id'], axis=1)
train_data = train_data.sort_values(by=['booking_id'])
# take only unique values
train_data = train_data.drop_duplicates(subset=['booking_id'])
Y_train = train_data['rating_score']

print(X_train.shape)
print(Y_train.shape)

In [ ]:
# use gridsearch cv to find best parameters for HistGradientBoostingRegressor
param_grid = {
    'learning_rate': [0.05],
    'max_iter': [500],
    'max_leaf_nodes': [31],
    'max_depth': [7],
    'l2_regularization': [0.2],
    'early_stopping': [False],
    'validation_fraction': [0.2],
    'loss': ['squared_error']
}

grid = GridSearchCV(estimator=HistGradientBoostingRegressor(), param_grid=param_grid, cv=5, n_jobs=5, verbose=3)
grid.fit(X_train, Y_train)

print(grid.best_params_)
best_params = grid.best_params_

# split train to train and validation
X_train_2, X_val, Y_train_2, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# create model
model = HistGradientBoostingRegressor(**best_params)
# fit model
model.fit(X_train_2, Y_train_2)

train_mse = mean_squared_error(Y_train_2, model.predict(X_train_2))
val_mse = mean_squared_error(Y_val, model.predict(X_val))

print('Train MSE: ', train_mse)
print('Validation MSE: ', val_mse)

In [ ]:
# use best parameters to train model
# use best params
best_params = grid.best_params_
# create model
model = HistGradientBoostingRegressor(**best_params)
# fit model
model.fit(X_train, Y_train)

train_mse = mean_squared_error(Y_train, model.predict(X_train))
print("train_mse: {}".format(train_mse))

test_booking_df = bookings_payment_df[bookings_payment_df['booking_id'].isin(test_data['booking_id'])]

# create X_test
test_booking_df = test_booking_df.sort_values(by=['booking_id'])
X_test = test_booking_df.drop(['booking_id'], axis=1)

Y_test_pred = model.predict(X_test)

# prepare submission file
submission = pd.DataFrame()
submission['booking_id'] = test_booking_df['booking_id']
submission['rating_score'] = Y_test_pred

# change ratings below 0 to 0 and above 5 to 5
submission['rating_score'] = submission['rating_score'].apply(lambda x: 1 if x < 1 else x)
submission['rating_score'] = submission['rating_score'].apply(lambda x: 5 if x > 5 else x)

submission.to_csv('okay-something-3.csv', index=False)
submission.describe()